## Instalaciones

In [2]:
# %pip install caas_jupyter_tools
%pip install openpyxl
#terminal: python -m pip install --user openpyxl

Note: you may need to restart the kernel to use updated packages.


c:\Users\marin\OneDrive\Desktop\UdeM\Nube\Clase_1_2_3\Clases\MLOps_UdM\.venv\Scripts\python.exe: No module named pip


## Librerias

In [2]:
import openpyxl
from collections import Counter
import re
import pandas as pd, unicodedata, re

In [ ]:
# Leer el archivo adjunto
df = pd.read_excel("Consolidado_GPON_Final-1.xlsx", engine='openpyxl')

# Asegurar que la columna exista
col = "OBSERVACION TERRENO"
if col not in df.columns:
    raise ValueError(f"No se encontró la columna '{col}'. Columnas disponibles: {df.columns.tolist()}")

# Normalizar texto
texts = df[col].dropna().astype(str).str.lower().str.replace(r"[^a-z0-9áéíóúñü\s]", " ", regex=True)

# Buckets de palabras clave basados en el contexto de telecomunicaciones
buckets = {
    "Portafolio": [
        "cambio equipo", "wifi", "decos", "canales", "canales premium", "cantidad", "sport wins", "HBO", "Megas", "velocidad", "velocidad contratada",
          "contraseña", "validacion general", "MAC", "CO creada", "portafolio de equipo"
    ],
    "Soporte": [
        "aprov", "aprovionamiento", "activar", "Aprovisionar", "Cam_Eq", "cambio equipo", "Cierre CBAPON", "Cod_Comp", "Configurar VLAN", "Contingencia",
          "Estabilizar Pon", "IP navegacion", "Puertos Lan", "Reenvio Registros", "validacion general", "validacion general", "Cambio ont", "cambio modem",
            "conexión en tr", "conexión en tr", "reaprovisiona", "reaprov", "aprovisionamiento", "activar", "activar decos", "instalar", "instalacion"


    ],
    "Infraestructura": [
        "Sin cobertura", "Red ocupada", "Nap Trocado", "Actualizar Infra", "informacion infraestructura", "infraestructura"

    ]
}

# Función heurística para asignar etiquetas
def suggest_label(text):
    scores = {k: 0 for k in buckets}
    for k, kws in buckets.items():
        for kw in kws:
            if kw in text:
                scores[k] += 1
    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else "Otros"

# Aplicar a todas las observaciones
labels_suggested = texts.apply(suggest_label)
label_counts = labels_suggested.value_counts().rename_axis("Etiqueta sugerida").reset_index(name="Frecuencia")

# Mostrar resultados resumidos
display("Propuesta de etiquetas heurísticas (telecomunicaciones)", label_counts.head(20))

# Exportar a Excel
# df.to_excel("Libro3_etiquetado.xlsx", index=False)


'Propuesta de etiquetas heurísticas (telecomunicaciones)'

,Etiqueta sugerida,Frecuencia
0,Otros,66188
1,Portafolio,24602
2,Soporte,21903
3,Infraestructura,18906


In [ ]:
# Leer archivo
df = pd.read_excel("Consolidado_GPON_Final-1.xlsx", engine='openpyxl')
col = "OBSERVACION TERRENO"
if col not in df.columns:
    raise ValueError(f"No se encontró la columna '{col}'. Columnas disponibles: {df.columns.tolist()}")

# --- Función de normalización ---
def normalize(txt):
    txt = str(txt).lower()
    txt = unicodedata.normalize("NFKD", txt)
    txt = "".join(ch for ch in txt if not unicodedata.combining(ch))
    return re.sub(r"[^a-z0-9ñü\s]", " ", txt)

# Normalizar texto de la columna
texts = df[col].dropna().astype(str).map(normalize)

# Normalizar buckets también
buckets = {
    "Portafolio": [
        "cambio equipo","wifi","decos","canales","canales premium","cantidad","sport wins","hbo","megas",
        "velocidad","velocidad contratada","contraseña","validacion general","mac","co creada","portafolio de equipo",
        "CO creada", "portafolio de equipo", "canal de win", "win+"

    ],
    "Soporte": [
        "aprov","aprovionamiento","activar","aprovisionar","cam_eq","cambio equipo","cierre cbapon","cod_comp",
        "configurar vlan","contingencia","estabilizar pon","ip navegacion","puertos lan","reenvio registros",
        "validacion general","cambio ont","cambio modem","conexion en tr","reaprovisiona","reaprov",
        "aprovisionamiento","activar decos","instalar","instalacion", "eliminar vlan", "crear vlan", "chuze la ont", 
        "chuzar ont", "chuze ont", "configurar puerto", "clave incorrecta", "modo bris", "aproviciona",
        "aprovicionamiento", "avilitar", "configurar línea", "c629", "estavilizar pon", "equipo quemado"

    ],
    "Infraestructura": [
        "sin cobertura","red ocupada","nap trocado","actualizar infra","informacion infraestructura","infraestructura"
    ]
}
buckets = {k: [normalize(w) for w in v] for k, v in buckets.items()}

# --- Asignar etiquetas ---
def suggest_label(text):
    scores = {k: 0 for k in buckets}
    for k, kws in buckets.items():
        for kw in kws:
            if kw in text:
                scores[k] += 1
    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else "Otros"

labels = texts.map(suggest_label)
label_counts = labels.value_counts().rename_axis("Categoría").reset_index(name="Frecuencia")

# Mostrar resumen
display("Agrupación por Categoría", label_counts.head(20))


'Agrupación por Categoría'

,Categoría,Frecuencia
0,Otros,49923
1,Soporte,39541
2,Portafolio,23321
3,Infraestructura,18814


In [ ]:
import pandas as pd, re, unicodedata

# 1) Leer archivo
df = pd.read_excel("Consolidado_GPON_Final-1.xlsx", engine='openpyxl')
col = "OBSERVACION TERRENO"
if col not in df.columns:
    raise ValueError(f"No se encontró la columna '{col}'. Columnas disponibles: {df.columns.tolist()}")

# 2) Normalización
def normalize(txt):
    txt = str(txt).lower()
    txt = unicodedata.normalize("NFKD", txt)
    txt = "".join(ch for ch in txt if not unicodedata.combining(ch))
    return re.sub(r"[^a-z0-9ñü\s]", " ", txt)

texts = df[col].dropna().astype(str).map(normalize)

# 3) Buckets de palabras clave (ya definidos)
buckets = {
    "Portafolio": [
        "cambio equipo","wifi","decos","canales","canales premium","cantidad","sport wins","hbo","megas",
        "velocidad","velocidad contratada","contraseña","validacion general","mac","co creada",
        "portafolio de equipo","canal de win","win+"
    ],
    "Soporte": [
        "aprov","aprovionamiento","activar","aprovisionar","cam_eq","cambio equipo","cierre cbapon",
        "cod_comp","configurar vlan","contingencia","estabilizar pon","ip navegacion","puertos lan",
        "reenvio registros","validacion general","cambio ont","cambio modem","conexion en tr","reaprovisiona",
        "reaprov","aprovisionamiento","activar decos","instalar","instalacion","eliminar vlan","crear vlan",
        "chuze la ont","chuzar ont","chuze ont","configurar puerto","clave incorrecta","modo bris",
        "aproviciona","aprovicionamiento","avilitar","configurar línea","c629","estavilizar pon","equipo quemado","equipo quemado"
    ],
    "Infraestructura": [
        "sin cobertura","red ocupada","nap trocado","actualizar infra",
        "informacion infraestructura","infraestructura"
    ]
}
buckets = {k: [normalize(w) for w in v] for k, v in buckets.items()}

# 4) Función de etiquetado
def suggest_label(text):
    scores = {k: 0 for k in buckets}
    for k, kws in buckets.items():
        for kw in kws:
            if kw in text:
                scores[k] += 1
    best = max(scores, key=scores.get)
    return best if scores[best] > 0 else "Otros"

# 5) Agregar columna de etiqueta al final
df["Etiqueta"] = df[col].map(lambda x: suggest_label(normalize(x)))
df["Etiqueta"].value_counts().rename_axis("Etiqueta").reset_index(name="Frecuencia")

# 6) Exportar dataset etiquetado
# df.to_excel("Consolidado_GPON_Final_etiquetado.xlsx", index=False)
# print("✅ Archivo exportado: Consolidado_GPON_Final_etiquetado.xlsx")

,Etiqueta,Frecuencia
0,Otros,49886
1,Soporte,39794
2,Portafolio,23123
3,Infraestructura,18814


In [5]:
df["Etiqueta"].value_counts().rename_axis("Etiqueta").reset_index(name="Frecuencia")

,Etiqueta,Frecuencia
0,Otros,49886
1,Soporte,39755
2,Portafolio,23162
3,Infraestructura,18814
